# Relative Humidity Plots (all venues)

Choose your venue from the dropdown menu.  Then you can use the slider below the plot to explore your data.  You can make the data "window" as small or as large as you want, and also slide the entire window to the left or the right.  You can also choose a window size to be an hour, a day, a week, or a year using the buttons above the plot.

These plots work best on a large screen.  If you can't see the right edge of the plot, try using your browser zoom features to get the full plot.  On many browsers, ctrl-+ (holding the control button and the + sign at the same time) will zoom in and ctrl-minus (the control button with the minus sign) will zoom out.  Zooming is also available from the browser menus. We are looking at what we can do to facilitate use on other screen sizes.


```{admonition} Bug work-around
If you change venue sometimes you will get the new data superimposed on the old data rather than replacing it.  If this happens, reload the page. 
```

In [ ]:
# Imports 
import ipywidgets as widgets
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go   
from IPython.display import display

# Get the possible data venues
venuekeysfile = "venue-keys.csv"
dfVenueKeys = pd.read_csv(venuekeysfile)
dfVenueKeys = dfVenueKeys.dropna(subset=['channel_id'])




#give user option to select their venue
venueDropdown = widgets.Dropdown(
    options=dfVenueKeys['venue_id'],
    value=dfVenueKeys['venue_id'][0],
    description='Venue ID:',
    disabled=False,
)


container = widgets.HBox(children=[venueDropdown])

print(venueDropdown.value)

#Retrieve the venue and begin graphing
dfCollatedDataSet = pd.DataFrame(columns=['timestamp', 'entry_id', 'temperature', 'rh', 'voltage', 'venue_id'])
for index, venueSensorDetails in dfVenueKeys.iterrows():
    
    sensorMacOfSelection = venueSensorDetails['sensor_MAC']
    venueOfSelection = str(venueSensorDetails['venue_id'])
    dfTempDataSet = pd.read_csv('deviceData/'+ "venue_" + venueOfSelection + "_with_device_" + sensorMacOfSelection + '.csv' )
    dfTempDataSet['timestamp'] = pd.to_datetime(dfTempDataSet['timestamp'])
    dfTempDataSet['venue_id'] = venueSensorDetails['venue_id']

    dfCollatedDataSet = dfCollatedDataSet.append(dfTempDataSet, ignore_index=True)
    dfCollatedDataSet['timestamp'] = pd.to_datetime(dfCollatedDataSet['timestamp'])
    print('Loading data for venue: ', venueSensorDetails['venue_id']) 


print('Check')
dfCollatedDataSet.sample(6)




# Assign an empty figure widget with two traces
trace0 = go.Scatter(customdata=dfCollatedDataSet[dfCollatedDataSet['venue_id'] == 0], 
                    y=dfCollatedDataSet['rh'], 
                    x = dfCollatedDataSet['timestamp'], 
                    mode='lines', 
                    hoverinfo='all', 
                    name='RH',
                    )

# trace1 = go.Scatter(customdata=dfCollatedDataSet[dfCollatedDataSet['venue_id'] == 0], 
#                     y=dfCollatedDataSet['rh'], 
#                     x = dfCollatedDataSet['timestamp'], 
#                     mode='lines', 
#                     hoverinfo='all', 
#                     name='Relative Humidity',

# )


g = go.FigureWidget(data=trace0,
                    #data=[trace0, trace1],
                    layout = go.Layout(
                        yaxis=dict(range=[0,0])           
                    ))


print("Job Done")



In [ ]:
updatemenu = []
buttons = []

# button with one option for each dataframe
for index, venue in dfVenueKeys.iterrows():
    buttons.append(dict(method='update',
                        label='Venue ' + str(venue['venue_id']),
                        visible=True,
                        args=[{'y':[dfCollatedDataSet[dfCollatedDataSet['venue_id']==venue['venue_id']]['rh'].values], #, 
                                    #dfCollatedDataSet[dfCollatedDataSet['venue_id']==venue['venue_id']]['rh'].values],
                               'x':[dfCollatedDataSet[dfCollatedDataSet['venue_id']==venue['venue_id']]['timestamp'].values],
                               'type':'scatter',   
                            #  'name': 'Temperature',
                                                          
                               },
                               {
                                   'title.text': 'RH for Venue = ' + str(venue['venue_id']),
                                   'title.font.color': 'green',
                                   #'yaxis.range': [-5,dfCollatedDataSet[dfCollatedDataSet['venue_id']==venue['venue_id']][['temperature', 'rh']].max().max()+5], 
                                   #'yaxis.range': [dfCollatedDataSet[dfCollatedDataSet['venue_id']==venue['venue_id']]['rh'].min()-5,dfCollatedDataSet[dfCollatedDataSet['venue_id']==venue['venue_id']]['rh'].max()+5], 
                                   # fixed, because we get rogue readings on cheap devices
                                   'yaxis.range': [0,100],  
                                   'yaxis.title.text': 'RH',
                                   'xaxis.title.text': 'Timestamp'

                               },
                               ],
                        )
                  )

# some adjustments to the updatemenus
updatemenu = []
your_menu = dict()
updatemenu.append(your_menu)

updatemenu[0]['buttons'] = buttons
updatemenu[0]['direction'] = 'down'
updatemenu[0]['showactive'] = True
# updatemenu[0]['active'] = 0

 

fig = go.Figure(g)
# add dropdown menus to the figure
fig.update_layout(showlegend=True, 
              updatemenus=updatemenu, 
              autosize = True, 
              title= "Please select a venue to see your data.",
              width=1000,
              height=500,
)

fig.update_layout(
    hovermode='x unified',
    hoverlabel=dict(
        bgcolor="white",
        # font_size=16,
        font_family="Rockwell"
    )
)

# Add range slider
fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(
                     label="All",
                     step="all"
                     ),
                                dict(count=1,
                     label="Hour",
                     step="hour",
                     stepmode="todate"),
                dict(count=1,
                     label="Day",
                     step="day",
                     stepmode="backward"),
                dict(count=7,
                     label="Week",
                     step="day",
                     stepmode="backward"),
                dict(count=1,
                     label="Year",
                     step="year",
                     stepmode="backward")
            ])
        ),
        rangeslider=dict(
            visible=True
        ),
        type="date"
    )
)


#fig.update_yaxes(range=[50, 60])  


#fig.add_hline(y=16, annotation_text='16C - usual minimum for children', annotation_font_color="blue", line_color='red', layer='above', line_dash='dash')
# fig.update_yaxes(range = [-5, dfCollatedDataSet['temperature'].max()+5])
fig.show()
